In [1]:
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Dropout
from tqdm import tqdm
%matplotlib inline

2024-01-19 16:30:20.269412: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-19 16:30:20.349161: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 16:30:20.349224: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 16:30:20.349286: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 16:30:20.366333: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-19 16:30:20.367909: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
dict_2classes = {}
dict_2classes['DDoS-RSTFINFlood'] = 'Attack'
dict_2classes['DDoS-PSHACK_Flood'] = 'Attack'
dict_2classes['DDoS-SYN_Flood'] = 'Attack'
dict_2classes['DDoS-UDP_Flood'] = 'Attack'
dict_2classes['DDoS-TCP_Flood'] = 'Attack'
dict_2classes['DDoS-ICMP_Flood'] = 'Attack'
dict_2classes['DDoS-SynonymousIP_Flood'] = 'Attack'
dict_2classes['DDoS-ACK_Fragmentation'] = 'Attack'
dict_2classes['DDoS-UDP_Fragmentation'] = 'Attack'
dict_2classes['DDoS-ICMP_Fragmentation'] = 'Attack'
dict_2classes['DDoS-SlowLoris'] = 'Attack'
dict_2classes['DDoS-HTTP_Flood'] = 'Attack'

dict_2classes['DoS-UDP_Flood'] = 'Attack'
dict_2classes['DoS-SYN_Flood'] = 'Attack'
dict_2classes['DoS-TCP_Flood'] = 'Attack'
dict_2classes['DoS-HTTP_Flood'] = 'Attack'


dict_2classes['Mirai-greeth_flood'] = 'Attack'
dict_2classes['Mirai-greip_flood'] = 'Attack'
dict_2classes['Mirai-udpplain'] = 'Attack'

dict_2classes['Recon-PingSweep'] = 'Attack'
dict_2classes['Recon-OSScan'] = 'Attack'
dict_2classes['Recon-PortScan'] = 'Attack'
dict_2classes['VulnerabilityScan'] = 'Attack'
dict_2classes['Recon-HostDiscovery'] = 'Attack'

dict_2classes['DNS_Spoofing'] = 'Attack'
dict_2classes['MITM-ArpSpoofing'] = 'Attack'

dict_2classes['BenignTraffic'] = 'Benign'

dict_2classes['BrowserHijacking'] = 'Attack'
dict_2classes['Backdoor_Malware'] = 'Attack'
dict_2classes['XSS'] = 'Attack'
dict_2classes['Uploading_Attack'] = 'Attack'
dict_2classes['SqlInjection'] = 'Attack'
dict_2classes['CommandInjection'] = 'Attack'

dict_2classes['DictionaryBruteForce'] = 'Attack'

In [3]:
#df = pd.read_csv('./filtered_data2.csv')

In [4]:
file_paths = glob.glob('/home/haohao/Downloads/archive/wataiData/csv/CICIoT2023/*.csv')
file_paths.sort()
print('file_paths len: ', len(file_paths))
df = pd.DataFrame()
for file_path in tqdm(file_paths[:round(len(file_paths) * 0.3)], desc='Processing files', unit='file'):
    dff = pd.read_csv(file_path)
    df = pd.concat([df, dff], ignore_index=True)

file_paths len:  169


Processing files: 100%|██████████| 51/51 [02:51<00:00,  3.36s/file]


In [5]:
df['label'] = df['label'].map(dict_2classes)
df = df.sample(frac=1, random_state=42)

In [6]:
df.head(10)

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
5279460,0.000000,54.00,6.00,64.00,11.306346,11.306346,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334454e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack
6663301,0.145200,217.75,6.99,63.59,29.517406,29.517406,0.0,0.0,1.0,0.0,...,9.031879,66.19,8.297354e+07,9.5,10.885608,12.765440,541.688358,0.29,141.55,Attack
8559664,0.008476,13.74,1.42,66.31,12.492760,12.492760,0.0,0.0,0.0,0.0,...,549.066680,920.22,8.325338e+07,9.5,41.775523,776.392424,317509.558997,0.95,141.55,Attack
6299347,0.000000,54.00,6.00,64.00,60.940691,60.940691,0.0,1.0,0.0,1.0,...,0.000000,54.00,8.334495e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack
9878162,0.000000,54.00,6.00,64.00,31.855998,31.855998,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.297255e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack
5980310,0.000000,54.00,6.00,64.00,4.574341,4.574341,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.306855e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack
9817752,0.000000,160.38,16.83,63.36,0.473485,0.473485,0.0,0.0,0.0,0.0,...,1.959534,160.98,8.301122e+07,9.5,17.971541,2.778919,48.451831,0.08,141.55,Attack
8683320,0.000000,0.00,1.00,64.00,36.340593,36.340593,0.0,0.0,0.0,0.0,...,0.000000,42.00,8.314999e+07,9.5,9.165151,0.000000,0.000000,0.00,141.55,Attack
5100457,0.000000,54.00,6.00,64.00,5.317224,5.317224,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.306729e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack
8295876,0.074212,65.20,6.00,64.00,3.375920,3.375920,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.298606e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,Attack


In [7]:
y = pd.get_dummies(df['label']).iloc[:, ::-1].to_numpy()

y_labels = np.argmax(y, axis=1)

y[0:10]

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint8)

In [8]:

# Đếm số lượng mỗi nhãn
label_counts = np.bincount(y_labels)

# In ra số lượng mỗi nhãn
for i, count in enumerate(label_counts):
    print(f"Label {i}: {count} occurrences")

Label 0: 308331 occurrences
Label 1: 12784839 occurrences


In [9]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(method='ffill', inplace=True)
scaler = MinMaxScaler()
X = df.iloc[:, :-1]
X = scaler.fit_transform(X)

In [10]:
X[0:10]

array([[0.00000000e+00, 5.45159430e-06, 1.27659574e-01, 2.50980392e-01,
        1.34782152e-06, 1.34782152e-06, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.35135135e-01, 0.00000000e+00,
        1.21315055e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 4.52541733e-03, 2.06327373e-03, 2.87273772e-04,
        1.03819930e-03, 0.00000000e+00, 9.19117647e-04, 4.97165449e-01,
        6.07142857e-01, 9.00826420e-03, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 5.76970443e-01],
       [1.46024390e-06, 2.19830492e-05, 1.48723404e-01, 2.49372549e-01,
        3.51874905e-06, 3.51874905e-06, 0.00000000e+00, 0.00000000e+00,
        1.00000000e+00,

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=75)

In [12]:
y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

In [13]:

# Đếm số lượng mỗi nhãn
label_counts = np.bincount(y_test_labels)
#label_counts = np.bincount(y_train_labels)

# In ra số lượng mỗi nhãn
for i, count in enumerate(label_counts):
    print(f"Label {i}: {count} occurrences")

Label 0: 61733 occurrences
Label 1: 2556901 occurrences


In [14]:
from sklearn.metrics import classification_report, roc_auc_score

def eval(model):
  # Dự đoán nhãn trên tập kiểm thử
  y_pred = model.predict(X_test)

  # Chuyển đổi dự đoán thành nhãn cụ thể
  y_pred_labels = np.argmax(y_pred, axis=1)
  y_true_labels = np.argmax(y_test, axis=1)

  # Tính toán F1-score, Recall và Precision
  report = classification_report(y_true_labels, y_pred_labels, target_names=None)
  print(report)


In [15]:
def build_model():
  n_features = X_train.shape[1]
  model = Sequential()
  model.add(Dense(512, input_dim=n_features, activation='relu'))  # Input layer và hidden layer đầu tiên
  model.add(Dropout(0.3))  # Thêm dropout để giảm overfitting
  model.add(Dense(256, activation='relu'))  # Thêm hidden layer thứ hai
  model.add(Dropout(0.3))  # Thêm dropout
  model.add(Dense(128, activation='relu'))  # Thêm hidden layer thứ hai
  model.add(Dropout(0.3))  # Thêm dropout
  model.add(Dense(64, activation='relu'))  # Thêm hidden layer thứ hai
  model.add(Dropout(0.3))  # Thêm dropout
  model.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer

  # Compile mô hình
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = build_model()

# Huấn luyện mô hình
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

eval(model)


2024-01-19 16:34:26.147046: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1927314624 exceeds 10% of free system memory.


Epoch 1/10
81833/81833 [==============================] - 968s 12ms/step - loss: 0.0192 - accuracy: 0.9907 - val_loss: 0.0168 - val_accuracy: 0.9918
Epoch 2/10
81833/81833 [==============================] - 938s 11ms/step - loss: 0.0175 - accuracy: 0.9921 - val_loss: 0.0165 - val_accuracy: 0.9922
Epoch 3/10
81833/81833 [==============================] - 928s 11ms/step - loss: 0.0173 - accuracy: 0.9924 - val_loss: 0.0158 - val_accuracy: 0.9932
Epoch 4/10
81833/81833 [==============================] - 967s 12ms/step - loss: 0.0171 - accuracy: 0.9925 - val_loss: 0.0154 - val_accuracy: 0.9934
Epoch 5/10
81833/81833 [==============================] - 958s 12ms/step - loss: 0.0170 - accuracy: 0.9926 - val_loss: 0.0155 - val_accuracy: 0.9933
Epoch 6/10
81833/81833 [==============================] - 908s 11ms/step - loss: 0.0172 - accuracy: 0.9926 - val_loss: 0.0152 - val_accuracy: 0.9935
Epoch 7/10
81833/81833 [==============================] - 718s 9ms/step - loss: 0.0173 - accuracy: 0.9927 

In [17]:
# dem so luong false / true

In [18]:
# Tính tổng số mẫu
n_samples = X_train.shape[0]

# Tính chỉ số cho việc chia tách
# Phần 1: 0 -> 5/10 * n_samples
# Phần 2: 5/10 * n_samples -> (5+3)/10 * n_samples
# Phần 3: (5+3)/10 * n_samples -> n_samples
indices = [int(5/10 * n_samples), int((5+3)/10 * n_samples)]

# Chia dữ liệu
client_data = np.split(X_train, indices)
client_labels = np.split(y_train, indices)

# client_data[0], client_labels[0] sẽ chứa 50% dữ liệu đầu tiên
# client_data[1], client_labels[1] sẽ chứa 30% dữ liệu tiếp theo
# client_data[2], client_labels[2] sẽ chứa 20% dữ liệu cuối cùng


In [19]:
def fed_avg(models, data_distribution):
    """
    models: danh sách các mô hình từ các node
    data_distribution: danh sách chứa số lượng dữ liệu mỗi node có
    """
    # Tính tổng số mẫu dữ liệu
    total_data = sum(data_distribution)

    # Khởi tạo trọng số trung bình
    avg_weights = [np.zeros_like(w) for w in models[0].get_weights()]

    # Cộng trọng số từ tất cả mô hình, đã được nhân với tỷ lệ dữ liệu tương ứng
    for model, data_count in zip(models, data_distribution):
        for idx, layer_weights in enumerate(model.get_weights()):
            avg_weights[idx] += (layer_weights * data_count) / total_data

    return avg_weights


In [20]:
global_model = build_model()

In [21]:
num_rounds = 50
num_epochs = 3
batch_size = 32

for round in range(num_rounds):
    local_models = []
    cnt = 1
    for client_X, client_y in zip(client_data, client_labels):
        print('Round ', round, ' - Client ', cnt)
        cnt += 1
        # Sao chép trọng số từ global model
        local_model = build_model()
        local_model.set_weights(global_model.get_weights())

        # Huấn luyện mô hình cục bộ
        local_model.fit(client_X, client_y, epochs=num_epochs, batch_size=batch_size, verbose=0)
        local_models.append(local_model)

    # Cập nhật global model dựa trên trung bình trọng số từ các mô hình cục bộ
    global_model.set_weights(fed_avg(local_models, [len(cli_dt) for cli_dt in client_data]))

    print('Round ', round, ' - Evaluation:')
    eval(global_model)

Round  0  - Client  1
Round  0  - Client  2
Round  0  - Client  3
Round  0  - Evaluation:
81833/81833 [==============================] - 139s 2ms/step
              precision    recall  f1-score   support

           0       0.05      1.00      0.10     61733
           1       1.00      0.57      0.72   2556901

    accuracy                           0.58   2618634
   macro avg       0.53      0.78      0.41   2618634
weighted avg       0.98      0.58      0.71   2618634

Round  1  - Client  1
Round  1  - Client  2
Round  1  - Client  3
Round  1  - Evaluation:
81833/81833 [==============================] - 139s 2ms/step
              precision    recall  f1-score   support

           0       0.63      0.32      0.42     61733
           1       0.98      1.00      0.99   2556901

    accuracy                           0.98   2618634
   macro avg       0.81      0.66      0.71   2618634
weighted avg       0.98      0.98      0.98   2618634

Round  2  - Client  1
Round  2  - Client  2
